# SH_cuts
This notebook analyzes muon and neutrino reconstruction performance for Solid Hydrogen events using ROOT (PyROOT) and RDataFrame.

It loads a ROOT file (`Rough_tree`), applies filters based on reconstruction quality (reco_flag), event topology (true_flag), and track multiplicity (track_flag), then produces comprehensive histograms and plots showing:
- **Muon properties**: momentum components (x,y,z), total momentum, and energy residuals
- **Reconstruction performance**: comparison of true vs reconstructed quantities
- **Neutrino residuals**: energy and momentum components derived from muon properties and beam direction
- **Resolution studies**: momentum resolution as a function of number of points and total momentum for single-track events
- **2D correlation plots**: reco vs true momentum components

Results are saved to `Plots/SH_mu_nu_reco_performance/SH_reco_performance.root` with detailed histograms and PDF plots.

Dependencies: ROOT (PyROOT), RDataFrame, and the local helper module `df_functions_utils.py`.

In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
from ROOT import gStyle
import df_functions_utils as df_utils

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Plots/SH_cuts/"

In [ ]:
### Uncoment for fast testing
df = df.Range(1000)

### Generating output file

In [ ]:
f_out = ROOT.TFile(Out_dir + "SH_cuts.root", "RECREATE")

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_extend =  df + additional columns needed in the analysis 
- df_flag = df_extended + flags that identify the filters  

### Filters flags for df_flag:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_extended = df_utils.df_extend(df)
df_flag = df_utils.df_addflags(df_extended)

# Plotting

GOOD RECO SELECTION

In [ ]:
#selected events after good reco cut on POLYPROPYLENE


nbin = 12

df_plasticR = df_flag.Filter('reco_flag == 1 && Where_int == "C3H6"')

hist_R1 = df_plasticR.Filter('target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"').Histo1D(("hist_H_QES_goodreco", "", nbin, 0, 6000),"nu_Ereco")
hist_R2 = df_plasticR.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco", "", nbin, 0, 6000),"nu_Ereco")
hist_R3 = df_plasticR.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Histo1D(("hist_H_nonQES_goodreco", "", nbin, 0, 6000),"nu_Ereco")

hist_R1.SetFillColor(ROOT.kBlue)
hist_R2.SetFillColor(ROOT.kRed)
hist_R3.SetFillColor(ROOT.kGreen)


stackR = ROOT.THStack("stack_goodreco", " ")
stackR.Add(hist_R1.GetPtr())
stackR.Add(hist_R2.GetPtr())
stackR.Add(hist_R3.GetPtr())

cR = ROOT.TCanvas()
cR.SetLeftMargin(0.15)

stackR.Draw("HIST")
stackR.GetXaxis().SetTitle(" Reconstructed neutrino energy [MeV]")
stackR.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)

legend.AddEntry(hist_R1.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_R2.GetPtr(), "CC on C in plastic", "f")
legend.AddEntry(hist_R3.GetPtr(), "CC non QES on H in plastic", "f")


legend.Draw()

cR.Draw()
cR.SaveAs(Out_dir +"reco_cut.pdf")
f_out.cd()

hist_R1.GetValue().Write()
hist_R2.GetValue().Write()
hist_R3.GetValue().Write()
stackR.Write()



EVENTS WITH ONE TRACK

In [ ]:
df_one_track = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")

In [ ]:
#selected events after track cut on POLYPROPYLENE

nbin = 12

df_plastic = df_one_track.Filter('Where_int == "C3H6"')

hist_1T = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"').Histo1D(("hist_H_QES_goodreco_1T", "", nbin, 0, 6000),"nu_Ereco")
hist_2T = df_plastic.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco_1T", "", nbin, 0, 6000),"nu_Ereco")
hist_3T = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Histo1D(("hist_H_nonQES_goodreco_1T", "", nbin, 0, 6000),"nu_Ereco")

hist_1T.SetFillColor(ROOT.kBlue)
hist_2T.SetFillColor(ROOT.kRed)
hist_3T.SetFillColor(ROOT.kGreen)


stackT = ROOT.THStack("stack_goodreco_1T", " ")
stackT.Add(hist_1T.GetPtr())
stackT.Add(hist_2T.GetPtr())
stackT.Add(hist_3T.GetPtr())

cT = ROOT.TCanvas()
cT.SetLeftMargin(0.15)

stackT.Draw("HIST")
stackT.GetXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
stackT.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)


legend.AddEntry(hist_1T.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_2T.GetPtr(), "CC on C in plastic", "f")
legend.AddEntry(hist_3T.GetPtr(), "CC non QES on H in plastic", "f")

legend.Draw()

cT.Draw()
cT.SaveAs(Out_dir +"track_cut.pdf")
f_out.cd()

hist_1T.GetValue().Write()
hist_2T.GetValue().Write()
hist_3T.GetValue().Write()
stackT.Write()

NEUTRON SPACE SELECTION

In [ ]:
nbin = 12

df_space = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1")

df_plastic = df_space.Filter('Where_int == "C3H6"')

hist_1 = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"').Histo1D(("hist_H_QES_goodreco_1T_ECALs", "", nbin, 0, 6000),"nu_Ereco")
hist_2 = df_plastic.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco_1T_ECALs", "", nbin, 0, 6000),"nu_Ereco")
hist_3 = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Histo1D(("hist_H_nonQES_goodreco_1T_ECALs", "", nbin, 0, 6000),"nu_Ereco")


hist_1.SetFillColor(ROOT.kBlue)
hist_2.SetFillColor(ROOT.kRed)
hist_3.SetFillColor(ROOT.kGreen)


stack = ROOT.THStack("stack_goodreco_1T_ECALs", " ")
stack.Add(hist_1.GetPtr())
stack.Add(hist_2.GetPtr())
stack.Add(hist_3.GetPtr())

c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack.Draw("HIST")
stack.GetXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
stack.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)

legend.AddEntry(hist_1.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_2.GetPtr(), "CC on C in plastic", "f")
legend.AddEntry(hist_3.GetPtr(), "CC non QES on H in plastic", "f")

legend.Draw()

c.Draw()

c.SaveAs(Out_dir + "space_cut.pdf")
f_out.cd()

hist_1.GetValue().Write()
hist_2.GetValue().Write()
hist_3.GetValue().Write()
stack.Write()


In [ ]:
### Equivalent in graphite
nbin = 20

df_graf = df_space.Filter('Where_int == "Graphite"')

hist_2 = df_graf.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco_1T_ECALs_Graphite", "", nbin, 0, 10000),"nu_Ereco")
hist_2.SetFillColor(ROOT.kRed)


stack = ROOT.THStack("stack_goodreco_1T_ECALs_Graphite", " ")
stack.Add(hist_2.GetPtr())

c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack.Draw("HIST")
stack.GetXaxis().SetTitle("Reco Neutrino Energy [MeV]")
stack.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.55,0.45,0.9,0.9)
legend.AddEntry(hist_2.GetPtr(), "CC on C in graphite", "f")

legend.Draw()

c.Draw()
c.SaveAs(Out_dir +"space_cut_graphite.pdf")
f_out.cd()
hist_2.GetValue().Write()
stack.Write()

In [ ]:
nbin = 20

df_spaceTRUE = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && spaceTRUE_flag == 1")

df_plasticTRUE = df_spaceTRUE.Filter('Where_int == "C3H6"')

hist_1 = df_plasticTRUE.Filter('target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"').Histo1D(("hist_H_QES_goodreco_1T_ECALsMC", "", nbin, 0, 10000),"nu_Ereco")
hist_2 = df_plasticTRUE.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco_1T_ECALsMC", "", nbin, 0, 10000),"nu_Ereco")
hist_3 = df_plasticTRUE.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Histo1D(("hist_H_nonQES_goodreco_1T_ECALsMC", "", nbin, 0, 10000),"nu_Ereco")

hist_1.SetFillColor(ROOT.kBlue)
hist_2.SetFillColor(ROOT.kRed)
hist_3.SetFillColor(ROOT.kGreen)


stack = ROOT.THStack("stack_goodreco_1T_ECALsMC", " ")
stack.Add(hist_1.GetPtr())
stack.Add(hist_2.GetPtr())
stack.Add(hist_3.GetPtr())

c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack.Draw("HIST")
stack.GetXaxis().SetTitle("Reco Neutrino Energy [MeV]")
stack.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.55,0.45,0.9,0.9)

legend.AddEntry(hist_1.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_2.GetPtr(), "CC on C in plastic", "f")
legend.AddEntry(hist_3.GetPtr(), "CC non QES on H in plastic", "f")

legend.Draw()

c.Draw()
c.SaveAs(Out_dir +"space_cut_MC.pdf")
f_out.cd()
hist_1.GetValue().Write()
hist_2.GetValue().Write()
hist_3.GetValue().Write()
stack.Write()

NEUTRON TIME SELECTION

In [ ]:
df_time = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1")

In [ ]:
nbin = 12

df_plastic = df_time.Filter('Where_int == "C3H6"')

hist_1 = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"').Histo1D(("hist_H_QES_goodreco_1T_ECALs_T", "", nbin, 0, 6000),"nu_Ereco")
hist_2 = df_plastic.Filter('target == "tgt:1000060120"').Histo1D(("hist_C_goodreco_1T_ECALs_T", "", nbin, 0, 6000),"nu_Ereco")
hist_3 = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Histo1D(("hist_H_nonQES_goodreco_1T_ECALs_T", "", nbin, 0, 6000),"nu_Ereco")


hist_1.SetFillColor(ROOT.kBlue)
hist_2.SetFillColor(ROOT.kRed)
hist_3.SetFillColor(ROOT.kGreen)

stack = ROOT.THStack("stack_goodreco_1T_ECALs_T", " ")
stack.Add(hist_1.GetPtr())
stack.Add(hist_2.GetPtr())
stack.Add(hist_3.GetPtr())


c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack.Draw("HIST")
stack.GetXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
stack.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)

legend.AddEntry(hist_1.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_2.GetPtr(), "CC on C in plastic", "f")
legend.AddEntry(hist_3.GetPtr(), "CC non QES on H in plastic", "f")

legend.Draw()


c.Draw()
c.SaveAs(Out_dir + "time_cut.pdf")
f_out.cd()
hist_1.GetValue().Write()
hist_2.GetValue().Write()
hist_3.GetValue().Write()
stack.Write()

PERCENTAGES EVOLUTION CUT BY CUT

In [ ]:
total = df_plastic.Count().GetValue()

CCnonQEH = df_plastic.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES" ').Count().GetValue() #green
CConC = df_plastic.Filter('target == "tgt:1000060120"').Count().GetValue()
CCnonQEC = df_plastic.Filter('target == "tgt:1000060120" && st_proc_type != "proc:Weak[CC],QES" ').Count().GetValue() #red
mec = df_plastic.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],MEC" ').Count().GetValue()
dis = df_plastic.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],DIS" ').Count().GetValue()
res = df_plastic.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],RES" ').Count().GetValue()


print("CC non QE on H:")
print(total)
print(CCnonQEH)
print(CCnonQEH/total)

print("-------------------------")
print("CCnonQE on C")
print(CConC)
print(CCnonQEC/CConC)


print("--------------MEC-----------")
print(mec)
print(mec/CCnonQEC)

print("--------------DIS-----------")
print(dis)
print(dis/CCnonQEC)

print("--------------RES-----------")
print(res)
print(res/CCnonQEC)

In [ ]:
df_plastic.Filter('target == "tgt:1000010010" && st_proc_type != "proc:Weak[CC],QES"').Display(["st_proc_type", "primaries_PDG"],100).Print()

In [ ]:
df_plastic.Filter('target == "tgt:1000060120" && st_proc_type != "proc:Weak[CC],QES"').Display(["st_proc_type", "primaries_PDG"],100).Print()

In [ ]:
df_vol =  df_flag.Filter("volume_flag == 1")
df_reco =  df_flag.Filter("volume_flag == 1 && reco_flag == 1")
df_one_track =  df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")
df_space = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1")
df_time = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1")

evt_tot = [
    df.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]
evt_vol = [
    df_vol.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]
evt_reco = [
    df_reco.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]

evt_track = [
    df_one_track.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]
evt_space= [
    df_space.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]
evt_time= [
    df_time.Filter(f"Evt_cat == {i}").Count().GetValue() for i in range(6)
]

# Calcolo percentuali
print(f"{'Category':<10}{'Total':<10}{'Volume':<10}{'Removed (%)':<15}{'Good Reco':<15}{'Removed (%)':<15}{'1 track':<12} {'Removed (%)':<15}{'n space':<15}{'Removed (%)':<12}{'n time':<15}{'Removed (%)':<12}")
print("-" * 100)

for i in range(6):
    tot = evt_tot[i]
    vol = evt_vol[i]
    reco = evt_reco[i]
    one = evt_track[i]
    space = evt_space[i]
    time = evt_time[i]
    perc_removed1 = 0 if tot == 0 else (tot - vol) / tot * 100
    perc_removed2 = 0 if vol == 0 else (vol - reco) / vol * 100
    perc_removed3 = 0 if reco == 0 else (reco - one) / reco * 100
    perc_removed4 = 0 if one == 0 else (one - space) / one * 100
    perc_removed5 = 0 if space == 0 else (space - time) / space * 100
    
    print(f"{i:<10} {tot:<10}{vol:<10}{perc_removed1:<15.2f}{reco:<15}{perc_removed2:<15.2f}{one:<12} {perc_removed3:<15.2f}{space:<15} {perc_removed4:<12.2f}{time:<15} {perc_removed5:<12.2f}")

In [ ]:
#FATTO CON NUOVE CATEGORIE, GUARDARE FUNZIONE CAT


df_vol =  df_flag.Filter("volume_flag == 1")
df_reco =  df_flag.Filter("volume_flag == 1 && reco_flag == 1")
df_one_track =  df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")
df_space = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1")
df_time = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1")

evt_tot = [
    df_extended.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_vol = [
    df_vol.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_reco = [
    df_reco.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]

evt_track = [
    df_one_track.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_space= [
    df_space.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_time= [
    df_time.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]

# Calcolo percentuali
print(f"{'Category':<10}{'Total':<10}{'Volume':<10}{'Removed (%)':<15}{'Good Reco':<15}{'Removed (%)':<15}{'1 track':<12} {'Removed (%)':<15}{'n space':<15}{'Removed (%)':<12}{'n time':<15}{'Removed (%)':<12}")
print("-" * 100)

for i in range(6):
    tot = evt_tot[i]
    vol = evt_vol[i]
    reco = evt_reco[i]
    one = evt_track[i]
    space = evt_space[i]
    time = evt_time[i]
    perc_removed1 = 0 if tot == 0 else (tot - vol) / tot * 100
    perc_removed2 = 0 if vol == 0 else (vol - reco) / vol * 100
    perc_removed3 = 0 if reco == 0 else (reco - one) / reco * 100
    perc_removed4 = 0 if one == 0 else (one - space) / one * 100
    perc_removed5 = 0 if space == 0 else (space - time) / space * 100
    
    print(f"{i:<10} {tot:<10}{vol:<10}{perc_removed1:<15.2f}{reco:<15}{perc_removed2:<15.2f}{one:<12} {perc_removed3:<15.2f}{space:<15} {perc_removed4:<12.2f}{time:<15} {perc_removed5:<12.2f}")

In [ ]:

df_vol =  df_flag.Filter("volume_flag == 1")
df_reco =  df_flag.Filter("volume_flag == 1 && reco_flag == 1")
df_one_track =  df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")
df_space = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1")
df_time = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1")

evt_tot = [
    df_extended.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_vol = [
    df_vol.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_reco = [
    df_reco.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]

evt_track = [
    df_one_track.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_space= [
    df_space.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]
evt_time= [
    df_time.Filter(f"Cat == {i}").Count().GetValue() for i in range(6)
]

# Calcolo percentuali
print(f"{'Category':<10}{'Total':<10}{'Volume':<10}{'Removed (%)':<15}{'Good Reco':<15}{'Removed (%)':<15}{'1 track':<12} {'Removed (%)':<15}{'n space':<15}{'Removed (%)':<12}{'n time':<15}{'Removed (%)':<12}")
print("-" * 100)

for i in range(6):
    tot = evt_tot[i]
    vol = evt_vol[i]
    reco = evt_reco[i]
    one = evt_track[i]
    space = evt_space[i]
    time = evt_time[i]
    perc_removed1 = 0 if tot == 0 else (tot - vol) / tot * 100
    perc_removed2 = 0 if vol == 0 else (vol - reco) / vol * 100
    perc_removed3 = 0 if reco == 0 else (reco - one) / reco * 100
    perc_removed4 = 0 if one == 0 else (one - space) / one * 100
    perc_removed5 = 0 if space == 0 else (space - time) / space * 100
    
    print(f"{i:<10} {tot:<10}{vol:<10}{perc_removed1:<15.2f}{reco:<15}{perc_removed2:<15.2f}{one:<12} {perc_removed3:<15.2f}{space:<15} {perc_removed4:<12.2f}{time:<15} {perc_removed5:<12.2f}")

In [ ]:
#CUT PLOT
N_tot = df.Count().GetValue()
N_vol = df_vol.Count().GetValue()
N_reco = df_reco.Count().GetValue()
N_track = df_one_track.Count().GetValue()
N_space = df_space.Count().GetValue()
N_time = df_time.Count().GetValue() 


labels = ["Total", "volume", "good reco", "one track", "neutron space match", "neutron time match"]
values = [N_tot, N_vol, N_reco, N_track,N_space,N_time]

n_bins = len(values)
hist_cut = ROOT.TH1F("hist_cut", " ", n_bins, 0.5, n_bins + 0.5)

for i, (label, val) in enumerate(zip(labels, values), start=1):
    hist_cut.SetBinContent(i, val)
    hist_cut.GetXaxis().SetBinLabel(i, label)

hist_cut.SetFillColor(ROOT.kBlue)
hist_cut.SetBarWidth(0.6)
hist_cut.SetBarOffset(0.2)
# hist.GetXaxis.SetTitle("selection cuts")
hist_cut.GetYaxis().SetTitle("entries")

c = ROOT.TCanvas("c", " ", 800, 600)
ROOT.gStyle.SetOptStat(0)
hist_cut.Draw("bar")
c.Draw()


f_out.cd()
hist_cut.Write()
c.SaveAs(Out_dir +"hist_cut.pdf")

## Statistical subtraction plots

In [ ]:
nbin = 12

h_C_graf = df_time.Filter('Where_int == "Graphite" && target == "tgt:1000060120"').Histo1D(("h_C_graf", "h_C_graf", nbin, 0, 6000),"nu_Ereco")

h_C_pl = df_time.Filter('Where_int == "C3H6" && target == "tgt:1000060120"').Histo1D(("h_C_pl", "h_C_pl", nbin, 0, 6000),"nu_Ereco")


h_C_graf.GetPtr().Scale(5.25)

h_C_pl.GetPtr().SetLineColor(ROOT.kRed)
h_C_graf.GetPtr().SetLineColor(ROOT.kBlack)
h_C_graf.GetPtr().SetMarkerStyle(20)


ROOT.gStyle.SetOptStat(0)

cc = TCanvas("c", " ", 900, 1200)
#cc = ROOT.TCanvas()
cc.SetLeftMargin(0.15)
cc.Divide(1,2)


cc.cd(1)

h_C_pl.GetPtr().GetXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
h_C_pl.GetPtr().GetYaxis().SetTitle("Entries")

h_C_pl.GetPtr().SetFillColor(kRed)

h_C_pl.Draw("HIST E")
h_C_graf.Draw("SAME")

legend = ROOT.TLegend(0.6,0.6,0.9,0.9)
legend.AddEntry(h_C_pl.GetPtr(), "CC on C in plastic", "l")
legend.AddEntry(h_C_graf.GetPtr(), "CC on C in graphite", "l")
# legend.Draw()

cc.cd(2)


h_ratio = h_C_graf.GetPtr().Clone("h_ratio")
h_ratio.Divide(h_C_pl.GetPtr())

h_ratio.SetLineColor(ROOT.kBlue)
h_ratio.SetTitle("")

h_ratio.GetYaxis().SetTitle("ratio")
h_ratio.GetXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
h_ratio.GetXaxis().SetRangeUser(0, 6000)
h_ratio.GetYaxis().SetRangeUser(0, 2.5)

h_ratio.Draw()

cc.Draw()
cc.SaveAs(Out_dir + "evt_on_C.pdf")

f_out.cd()
h_C_graf.GetValue().Write()
h_C_pl.GetValue().Write()
h_ratio.Write()


In [ ]:

nbin = 12

h_C_graf = df_time.Filter('Where_int == "Graphite" && target == "tgt:1000060120"') \
                   .Histo1D(("h_C_graf", "", nbin, 0, 6000), "nu_Ereco")

h_C_pl = df_time.Filter('Where_int == "C3H6" && target == "tgt:1000060120"') \
                 .Histo1D(("h_C_pl", "", nbin, 0, 6000), "nu_Ereco")


h_C_graf.GetPtr().Scale(5.25)


h_C_pl.GetPtr().SetLineColor(ROOT.kRed)
h_C_pl.GetPtr().SetFillColor(ROOT.kRed)
h_C_graf.GetPtr().SetLineColor(ROOT.kBlack)


cRP = ROOT.TCanvas("c", "RatioPlot", 600, 600)

rp = ROOT.TRatioPlot(h_C_pl.GetPtr(), h_C_graf.GetPtr())
rp.Draw()


rp.GetUpperRefYaxis().SetTitle("entries")
rp.GetLowerRefYaxis().SetTitle("ratio")
rp.GetLowerRefXaxis().SetTitle("Reconstructed neutrino energy [MeV]")
rp.GetUpperRefXaxis().SetTitle("Reconstructed neutrino energy [MeV]")

rp.GetUpperPad().SetLeftMargin(0.20)
rp.GetLowerPad().SetLeftMargin(0.20)

cRP.Update()
cRP.SaveAs(Out_dir + "C_evt.pdf")

f_out.cd()
h_C_graf.GetValue().Write()
h_C_pl.GetValue().Write()
rp.Write()

## Cell to close the file 

In [ ]:
f_out.Close()